**Synchronize to Google Drive**

This feature makes the google colab to connect to the google drive with mounting GDrive and defining the path.

In [1]:
# Synchronize to google drive, define the root path
import os

google_colab  = True
if google_colab == True:
  #This statement used to pointing the google drive storage 
  from google.colab import drive
  drive.mount('/content/gdrive')
  root_path = 'gdrive/My Drive/Colab Notebooks/'

  #This statement is purposed to import library from the Drive
  os.chdir('gdrive/My Drive/Colab Notebooks/')
  

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np
np.random.seed(0)
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional, GRU, Activation, Flatten
from tensorflow.keras.models import Model
#from keras.models import Model
#from keras.layers import Dense, Input, Dropout, LSTM, Activation, Flatten, GRU, Concatenate
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.callbacks import EarlyStopping
np.random.seed(1)

**Hyper Parameters**

In [0]:
gcn_hidden_layer = 3
hidden_size = 300
gru_units = 300
learning_rate = 0.001
batch_size = 1
clipnorm = 1.0
stop_early = True
embedding_size=300


**Loading the dataset**

Currently, The experiment is using the dataset from DUC2001 and DUC2002. It designed to support Extractive Summarization method in which extract all of documents through pre-processing and store all of sentences to dataset-duc2001 pkl and dataset-duc2002.pkl and each of sentence has labeled based on whether its sentence is similar or exist on the summaries (labeled 1) or no (labeled 0)

In [4]:
import nltk
import collections
import pickle #data preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
nltk.download('punkt')
NUM_WORDS=25000
counter = collections.Counter()
print('Dataset loading ...')
#import postprocessing as pr #helper
#Step 1 - Load data
#Step 1 - Load data
with open('datasets/dataset-duc2001.pkl', 'rb') as fp:
    data_2001 = pickle.load(fp)

with open('datasets/dataset-duc2002.pkl', 'rb') as fp:
    data_2002 = pickle.load(fp)

clusterSentences_2001, clusterDocuments_2001, clusterSummaries_2001, listY_2001 = data_2001
clusterSentences_2002, clusterDocuments_2002, clusterSummaries_2002, listY_2002 = data_2002

clusterSentences = clusterSentences_2001 + clusterSentences_2002
clusterDocuments = clusterDocuments_2001 + clusterDocuments_2002
listY = listY_2001 + listY_2002

# Extract to become one bucket of sentences
sentences = []
for clusterSentence in clusterSentences[:2]:
  for sentence in clusterSentence:
    sentences.append(sentence)

tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',lower=True)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
index_word = tokenizer.index_word
index_docs = tokenizer.index_docs
vocabularySize = len(word_index) + 1

#nWord = [len(sentence.split()) for sentence in sentences]
#nWord = sorted(nWord,  reverse=True)
#words = [sentence.split() for sentence in sentences]
#nWord = [len(sentence.split()) for sentence in sentences]
#maxlen = max(nWord)
sentenceLength = 200

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Dataset loading ...


In [5]:
print('total of words :', len(word_index))
print('total of sentences', len(sentences))
total_documents = 0
for docs in clusterDocuments[:2]:
  total_documents += len(docs)
print('total of documents', total_documents)
print('total of clusters', len(clusterSentences[:2]))

total of words : 5666
total of sentences 1790
total of documents 22
total of clusters 2


**Load Pre-trained Word2Vec**

We use gensim to load Word2Vec which is provided Google. It can be downloaded from https://code.google.com/archive/p/word2vec/

Based on the reference paper, the model use 300-dimensional pre-trained word2vec embeddings (Mikolov et al., 2013) as input to GRU$^{sent}$

In [6]:
from gensim.models import KeyedVectors
WORD2VEC_MODEL = "word-embeddings/word2vec/GoogleNews-vectors-negative300.bin.gz"
word2vec = KeyedVectors.load_word2vec_format(WORD2VEC_MODEL, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


**Define Y target**

Y target defined as categorical data but ideally it should be binary

In [7]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y
    
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
xs, ys = [], []
for sentence in sentences:
  words = [x.lower() for x in sentence.split()]
  wids = [word_index[word] for word in words]
  xs.append(wids)

ys = []
for clusterY in listY[:2]:
  for sentenceY in clusterY:
    ys.append(sentenceY)

X = pad_sequences(xs, maxlen=sentenceLength)
Y = np.array(ys)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.02, random_state=1)

#if we use categorical mode
Y_train_oh = convert_to_one_hot(Ytrain, C = 2)
Y_test_oh = convert_to_one_hot(Ytest, C = 2)


#create training and test data based on the clustering document
XClusterTrain, XClusterTest, YClusterTrain, YClusterTest = train_test_split(clusterSentences[:2], listY[:2], test_size=0.5, random_state=1)
print('XClusterTrain', len(XClusterTrain))
print('YClusterTrain', len(YClusterTrain))
print('XClusterTest', len(XClusterTest))
print('YClusterTest',len(XClusterTest))
### if want to reshape the target #############################################
#Y_train_oh = Y_train_oh.reshape(Y_train_oh.shape[0], Y_train_oh.shape[1], 1)
#Y_test_oh = Y_test_oh.reshape(Y_test_oh.shape[0], Y_test_oh.shape[1], 1)
#print(Y_train_oh[0])
#print(Y_test_oh[0])

XClusterTrain 1
YClusterTrain 1
XClusterTest 1
YClusterTest 1


Using TensorFlow backend.


**Embedding Layer**

This function is used to embed pre-trained word2vec as defined before, the function is to produce the matrix from all of the sentences which were mapped to word2vec

In [0]:
# GRADED FUNCTION: pretrained_embedding_layer
def pretrained_embedding_matrix(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    embedding_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        try:
          embedding_matrix[index, :] = word_to_vec_map[word]
        except:
          embedding_matrix[index, :] = 0

    return embedding_matrix

In [0]:
embedding_layer = pretrained_embedding_matrix(word2vec, word_index)

**GRU$^{sent}$**: Sentence Embedding

This model used to produce sentence embedding from a cluster document with $ N $sentences (s$_1$, s$_2$, ... , s$_N$) in total. for each sentence s$_i$ of $ L $ words (w$_1$, w$_2$, ..., w$_L$), GRU$^sent$ recurrently updates hidden states at each time step t:

h$_t ^{sent} = GRU^{sent} h_{t-1} ^{sent} . w_t $ ......... (4)

where w$_t$ is the word embedding for w$_t$, h$_t ^{sent}$ is the hidden state of GRU$^{sent}$. h$_0$ is initialized as a zero vector, and the input sentence embedding x$_i$ is the last hidden state :

x$_i = h_L ^{sent}$ ....................................... (5)

All sentence embeddings from the given document cluster are grouped as the node feature matrix X :

$X = 
\quad
\begin{bmatrix}
- X_1 - \\
- X_2 - \\
. \\
. \\
. \\
- X_N - 
\end{bmatrix}$ 



In [0]:
from tensorflow.keras.layers import Embedding
class GruSent(tf.keras.Model):
  def __init__(self, vocabulary_size, embedding_size, pretrained_embedding):
    super(GruSent, self).__init__()
    self.vocabulary_size = vocabulary_size
    self.embedding_size = embedding_size
    self.pretrained_embedding = pretrained_embedding
    self.batch_size = batch_size
    self.units = gru_units
    self.embedding = Embedding(self.vocabulary_size, self.embedding_size, weights=[self.pretrained_embedding], trainable=False)
    self.grusent = GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.dense = Dense(1, activation=tf.nn.relu)
    
  def call(self, x, hidden):
    x = self.embedding(x)
    output, state_h = self.grusent(x, initial_state = hidden)
    output = self.dense(output)
    return output, state_h
  def initialize_hidden_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

In [11]:
gruSentModel = GruSent(vocabularySize, embedding_size, embedding_layer)
gru_sent_h_0 = gruSentModel.initialize_hidden_state(Xtest.shape[0])
print('Xtest', Xtest.shape)
print('Initial Gru Sent Hidden', gru_sent_h_0.shape)
output, h_L = gruSentModel(Xtest, gru_sent_h_0)
print('output', output.shape)
print('h_L', h_L.shape)

Xtest (36, 200)
Initial Gru Sent Hidden (36, 300)
output (36, 200, 1)
h_L (36, 300)


**Cluster Embedding**

The model apply a second-level RNN $GRU^{doc}$ to encode the entire document cluster. Given a document cluster $C$ with $M$ documents $(d_1, d_2, ..., d_M)$, for document $d_i$ with $|d_i|$ sentences, $GRU^{doc}$ first builds the document embedding $d_i$ on top of sentence embeddings:

$h_t^{doc} = GRU^{doc}(h_{t-1}^{doc}, s_t)$ ...... (8)

$d_i = h_{d_i}^{doc} $ ..................................... (9)

where $s_t$ is the sentence embedding in the document d_i. In Eq. 9, we extract the last hidden state as the document embedding for $d_i$. 

In Eq. 10, we average over document embeddings to produce the cluster embedding $C$

$C = \frac1M $$\sum_{i=1}^M d_i$ ......................... (10)

All the GRU used are forward.


In [0]:
from tensorflow.keras.layers import Embedding
class GruDoc(tf.keras.Model):
  def __init__(self, vocabularySize, embedding_size):
    super(GruDoc, self).__init__()
    self.vocabulary_size = vocabularySize
    self.embedding_size = embedding_size
    self.units = gru_units
    self.embedding = Embedding(self.vocabulary_size, self.embedding_size, trainable=False)
    self.grudoc = GRU(self.units,
                      return_sequences=False,
                      return_state=False,
                      recurrent_initializer='glorot_uniform')
    self.dense = Dense(1, activation=tf.nn.relu)
    
  def call(self, x, hidden):
    # Should apply document embedding 
    x = self.embedding(x)
    output = self.grudoc(x, initial_state = hidden)
    output = self.dense(output)
    return output
  def initialize_hidden_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

In [13]:
#testing the model
gruDocModel = GruDoc(vocabularySize, embedding_size)
gru_doc_h_0 = gruDocModel.initialize_hidden_state(Xtest.shape[0])
print('Initial Gru Doc h_0', Xtest.shape)
print('gru Sent last Hidden', )
gru_doc_output = gruDocModel(h_L, gru_doc_h_0)
print('output', gru_doc_output.shape)

Initial Gru Doc h_0 (36, 200)
gru Sent last Hidden
output (36, 1)


In [14]:
import nltk
nltk.download('punkt')
def transformToVector(ClusterSentences):
  xs_cluster=[]
  for sentences in ClusterSentences:
    xs = []
    for sentence in sentences:
      words = [x.lower() for x in sentence.split()]
      wids = [word_index[word] for word in words]
      xs.append(wids)
    xs_pad = tf.keras.preprocessing.sequence.pad_sequences(xs, maxlen=sentenceLength, padding='post')  
    xs_cluster.append(xs_pad)
  return xs_cluster
XS_train = transformToVector(XClusterTrain)
YS_train = []
for clusterY in YClusterTrain:
  ys = []
  for sentenceY in clusterY:
    ys.append(sentenceY)
  YS_train.append(ys)
print(len(XS_train))
print(len(YS_train))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
1
1


In [15]:
XS_test = transformToVector(XClusterTest)
YS_test = []
for clusterY in YClusterTest:
  ys = []
  for sentenceY in clusterY:
    ys.append(sentenceY)
  YS_test.append(ys)
print(len(XS_test))
print(len(YS_test))

1
1


In [16]:
XS_train_sentences = []
for xs_cluster in XS_train:
  for xs_sentence in xs_cluster:
    XS_train_sentences.append(xs_sentence)

XS_test_sentences = []
for xs_cluster in XS_test:
  for xs_sentence in xs_cluster:
    XS_test_sentences.append(xs_sentence)

YS_train_labels = []
for ys_cluster in YS_train:
  for ys_label in ys_cluster:
    YS_train_labels.append(ys_label)

YS_test_labels = []
for ys_cluster in YS_test:
  for ys_label in ys_cluster:
    YS_test_labels.append(ys_label) 

print('XS_train_sentences', len(XS_train_sentences))
print('XS_test_sentences', len(XS_test_sentences))
print('YS_train_labels', len(YS_train_labels))
print('YS_test_labels', len(YS_test_labels)) 
YS_train_labels = tf.convert_to_tensor(YS_train_labels)
XS_train_sentences = tf.convert_to_tensor(XS_train_sentences)
XS_test_sentences = tf.convert_to_tensor(XS_test_sentences)
YS_test_labels = tf.convert_to_tensor(YS_test_labels)
print('XS train sentences shape', XS_train_sentences.shape)
print('YS train labels shape', YS_train_labels.shape)

XS_train_sentences 1106
XS_test_sentences 684
YS_train_labels 1106
YS_test_labels 684
XS train sentences shape (1106, 200)
YS train labels shape (1106,)


**Define The Optimazer and The Loss Function**

In [0]:
loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')

**Training**

In [0]:
@tf.function
def train_step(sentences, labels, initial_hidden):
  with tf.GradientTape() as tape:
    output, h_L = gruSentModel(sentences, initial_hidden)
    predictions = gruDocModel(h_L, initial_hidden)
    loss = loss_object(labels, predictions)
    
  gradients = tape.gradient(loss, gruDocModel.trainable_variables)
  optimizer.apply_gradients(zip(gradients, gruDocModel.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [0]:
@tf.function
def test_step(sentences, labels, initial_hidden):
  output, h_L  = gruSentModel(sentences, initial_hidden)
  predictions = gruDocModel(h_L, initial_hidden)
  t_loss = loss_object(labels, predictions)
  
  test_loss(t_loss)
  test_accuracy(labels, predictions)
  

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((XS_train_sentences, YS_train_labels)).batch(10)
test_ds = tf.data.Dataset.from_tensor_slices((XS_test_sentences, YS_test_labels)).batch(10)

In [21]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for sents, labels in train_ds:
    #print('sents', sents.shape)
    initial_hidden = gruSentModel.initialize_hidden_state(sents.shape[0])
    train_step(sents, labels, initial_hidden)

  for test_sentences, test_labels in test_ds:
    initial_hidden = gruSentModel.initialize_hidden_state(test_sentences.shape[0])
    test_step(test_sentences, test_labels, initial_hidden)
  

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # Reset the metrics for the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1, Loss: 0.5141651630401611, Accuracy: 96.66667938232422, Test Loss: 0.7824252247810364, Test Accuracy: 94.92755889892578
Epoch 2, Loss: 0.5141651630401611, Accuracy: 96.66667938232422, Test Loss: 0.7824252247810364, Test Accuracy: 94.92755889892578
Epoch 3, Loss: 0.5141651630401611, Accuracy: 96.66667938232422, Test Loss: 0.7824252247810364, Test Accuracy: 94.92755889892578
Epoch 4, Loss: 0.5141651630401611, Accuracy: 96.66667938232422, Test Loss: 0.7824252247810364, Test Accuracy: 94.92755889892578
Epoch 5, Loss: 0.5141651630401611, Accuracy: 96.66667938232422, Test Loss: 0.7824252247810364, Test Accuracy: 94.92755889892578


**Summarizing**

**Salience Scoring**

In [0]:
def clusterTransformToVector(ClusterSentences):
  xs=[]
  for sentences in ClusterSentences:
    words = [x.lower() for x in sentence.split()]
    wids = [word_index[word] for word in words]
    xs.append(wids)
  xs_pad = tf.keras.preprocessing.sequence.pad_sequences(xs, maxlen=sentenceLength, padding='post')
  return xs_pad
def salienceEstimation(clusterDocument):
  cluster_doc_vector = clusterTransformToVector(clusterDocument)
  print('cluster_doc_vector', cluster_doc_vector.shape)
  initial_hidden = gruSentModel.initialize_hidden_state(cluster_doc_vector.shape[0])
  print('initial_hidden', initial_hidden.shape)
  output, h_L = gruSentModel(cluster_doc_vector, initial_hidden)
  predictions = gruDocModel(h_L, initial_hidden)
  return predictions

In [46]:
salience_scores = salienceEstimation(clusterSentences[2])
print(salience_scores.shape)

cluster_doc_vector (393, 200)
initial_hidden (393, 300)
(393, 1)


**Sentences Selection And Summarization**